In [1]:
import time
import csv
import nltk
import re
import requests
import pandas as pd
from ast import literal_eval

import warnings
warnings.filterwarnings("ignore")

In [2]:
# only once run for download WordNet or update
import nltk
# nltk.download('wordnet', download_dir='./')
nltk.data.path.append('../')
from nltk.corpus import wordnet as wn

In [4]:
hypo = lambda s: s.hyponyms()
hyper = lambda s: s.hypernyms()

topsynset = wn.synset('hand_tool.n.01')
allsynset = list(topsynset.closure(hypo))
allsynset.append(topsynset)

allsynset

[Synset('awl.n.01'),
 Synset('bevel.n.02'),
 Synset('bodkin.n.03'),
 Synset('bodkin.n.04'),
 Synset('crank.n.04'),
 Synset('dibble.n.01'),
 Synset('file.n.04'),
 Synset('float.n.05'),
 Synset('graver.n.01'),
 Synset('gutter.n.04'),
 Synset('hammer.n.02'),
 Synset('hand_shovel.n.01'),
 Synset('marlinespike.n.01'),
 Synset('miter_box.n.01'),
 Synset('opener.n.03'),
 Synset('pallet.n.03'),
 Synset('pestle.n.03'),
 Synset('pick.n.06'),
 Synset('pincer.n.01'),
 Synset('pipe_cutter.n.01'),
 Synset('pitchfork.n.01'),
 Synset('plane.n.05'),
 Synset('pliers.n.01'),
 Synset('plumber's_snake.n.01'),
 Synset('plunger.n.03'),
 Synset('ravehook.n.01'),
 Synset('sandblaster.n.01'),
 Synset('saw.n.02'),
 Synset('scraper.n.01'),
 Synset('screwdriver.n.01'),
 Synset('shovel.n.01'),
 Synset('soldering_iron.n.01'),
 Synset('spatula.n.02'),
 Synset('spreader.n.01'),
 Synset('square.n.08'),
 Synset('straightedge.n.01'),
 Synset('tire_iron.n.01'),
 Synset('trowel.n.01'),
 Synset('weeder.n.02'),
 Synset('wire

In [52]:
import pandas as pd
import numpy as np

# inherit features from Gold Standard dataset
GS_all_agreed = pd.read_csv('../sampled_count/GS_All_Agreed.csv', index_col=0)
features_target = ['Synsets','domain_x',
                   'nrdirhypers_x',
                   'nrhypos_x',
                   'nrpartrels_normalised_x',
                   'depthfromtopsynset_normalised_x',
                   'glosslength_normalised_x',
                   'minwordlength_x',
                   'nroflemmas_x',
                   'polyscore_max_x',
                   'vote_x']
GS_adopt = GS_all_agreed[features_target]

GS_adopt

,Synsets,domain_x,nrdirhypers_x,nrhypos_x,nrpartrels_normalised_x,depthfromtopsynset_normalised_x,glosslength_normalised_x,minwordlength_x,nroflemmas_x,polyscore_max_x,vote_x
0,Synset('adjustable_wrench.n.01'),tool,1,7,0.0,1.012903,0.563173,17,2,1,nb
1,Synset('allen_wrench.n.01'),tool,1,0,0.0,1.012903,0.391092,12,1,1,nb
2,Synset('alligator_wrench.n.01'),tool,1,0,0.0,1.012903,1.517437,16,1,1,nb
3,Synset('awl.n.01'),tool,1,2,15.7,0.911613,0.985552,3,1,1,b
4,Synset('backsaw.n.01'),tool,1,0,0.0,1.114194,1.110701,7,2,1,nb
...,...,...,...,...,...,...,...,...,...,...,...
834,Synset('ballet_skirt.n.01'),garm,1,0,0.0,0.947552,0.578283,4,2,2,nb
835,Synset('mess_jacket.n.01'),garm,1,0,0.0,1.158120,1.652238,11,3,1,nb
836,Synset('long_johns.n.01'),garm,1,0,0.0,1.052836,0.479149,10,1,1,nb
837,Synset('undies.n.01'),garm,1,0,0.0,1.158120,0.280880,6,1,1,nb


In [39]:
# method to get the frequency of a synset in google ngrams
corpora = dict(eng_2019=26, eng_us_2012=17, eng_us_2009=5, eng_gb_2012=18, eng_gb_2009=6,
               chi_sim_2012=23, chi_sim_2009=11, eng_2012=15, eng_2009=0,
               eng_fiction_2012=16, eng_fiction_2009=4, eng_1m_2009=1,
               fre_2012=19, fre_2009=7, ger_2012=20, ger_2009=8, heb_2012=24,
               heb_2009=9, spa_2012=21, spa_2009=10, rus_2012=25, rus_2009=12,
               ita_2012=22)

def ngram_mean_by_year_sleep(synset, corpus, startYear, endYear, smoothing, caseInsensitive):
    global counter
    m = 0
    synset = wn.synset(synset[8:-2])
    for i in synset.lemmas():
        query = i.name()
        query = query.replace('_',' ') # recovery phrase from _
        params = dict(content=query, year_start=startYear, year_end=endYear,
                      corpus=corpora[corpus], smoothing=smoothing,
                      case_insensitive=caseInsensitive)
        if params['case_insensitive'] is False:
            params.pop('case_insensitive')
        if '?' in params['content']:
            params['content'] = params['content'].replace('?', '*')
        if '@' in params['content']:
            params['content'] = params['content'].replace('@', '=>')
        ## set a counter
        counter += 1
        if counter == 73:
            time.sleep(580)
            counter = 0
        req = requests.get('https://books.google.com/ngrams/graph', params=params)
        res = re.findall('ngrams.data = (.*?);\\n', req.text)
        if res:
            data = {qry['ngram']: qry['timeseries'] for qry in literal_eval(res[0])}
            df = pd.DataFrame(data)
            df.insert(0, 'year', list(range(startYear, endYear + 1)))
        else:
            df = pd.DataFrame()
        if df.shape[1] > 1: # if the query exists in google ngrams.
            ngramsum = df.iloc[:,1].sum(axis = 0, skipna = True) # sum of percentage of the lemmas each year
            # time.sleep(10) # to avoid defense from the google ngram server
            m += ngramsum
        else:
            m += 0 # irrelevant, obviously.
    return m/(endYear-startYear+1)

In [43]:
ngram_mean_by_year_sleep("Synset('ballet_skirt.n.01')", 'eng_2019', 2009, 2019, 0, True)

5.498722571118083e-07

In [53]:
GS_adopt_test = GS_adopt[0:100]
counter = 0
GS_adopt_test['ngram_2009_2019'] = GS_adopt_test['Synsets'].apply(lambda syn: ngram_mean_by_year_sleep(syn, 'eng_2019', 2009, 2019, 0, True))

GS_adopt_test

,Synsets,domain_x,nrdirhypers_x,nrhypos_x,nrpartrels_normalised_x,depthfromtopsynset_normalised_x,glosslength_normalised_x,minwordlength_x,nroflemmas_x,polyscore_max_x,vote_x,ngram_2009_2019
0,Synset('adjustable_wrench.n.01'),tool,1,7,0.0,1.012903,0.563173,17,2,1,nb,2.460615e-08
1,Synset('allen_wrench.n.01'),tool,1,0,0.0,1.012903,0.391092,12,1,1,nb,1.601034e-08
2,Synset('alligator_wrench.n.01'),tool,1,0,0.0,1.012903,1.517437,16,1,1,nb,1.236022e-10
3,Synset('awl.n.01'),tool,1,2,15.7,0.911613,0.985552,3,1,1,b,2.805608e-07
4,Synset('backsaw.n.01'),tool,1,0,0.0,1.114194,1.110701,7,2,1,nb,1.135172e-08
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Synset('round_file.n.01'),tool,1,0,0.0,1.012903,1.095058,10,1,1,nb,9.592480e-09
96,Synset('router_plane.n.01'),tool,1,0,0.0,1.012903,1.392288,12,1,1,nb,1.431694e-09
97,Synset('s_wrench.n.01'),tool,1,0,0.0,1.114194,0.500598,8,1,1,nb,1.298056e-09
98,Synset('saw.n.02'),tool,1,16,15.7,0.911613,0.688322,3,1,28,b,2.283962e-04


In [3]:
def request_limit():
    start = time.time()
    count = 0
    flag = 0
    while True:
        params = dict(content='awl', year_start=2009, year_end=2019,
                          corpus='eng_2019', smoothing=0,
                          case_insensitive=True)
        if params['case_insensitive'] is False:
            params.pop('case_insensitive')
        if '?' in params['content']:
            params['content'] = params['content'].replace('?', '*')
        if '@' in params['content']:
            params['content'] = params['content'].replace('@', '=>')
        req = requests.get('https://books.google.com/ngrams/graph', params=params)
        count += 1
        if req.status_code == 429 and flag == 0:
            end = time.time()
            limit_start = time.time()
            period = end - start
            print("start limit")
            print(count, period)
            count = 0
            flag = 1
            print("flag = " + str(flag) + '\n')
        if req.status_code == 200 and flag == 1:
            print("limit end")
            limit_end = time.time()
            limit = limit_end - limit_start
            print('limit: ' + str(limit))
            count = 0
            flag = 0
            start = time.time()
            print('flag = ' + str(flag) + '\n')

    return

In [4]:
request_limit()

start limit
76 34.91451811790466
flag = 1

limit end
limit: 564.857830286026
flag = 0

start limit
75 32.35754442214966
flag = 1

limit end
limit: 567.7605030536652
flag = 0

start limit
75 34.245885372161865
flag = 1

limit end
limit: 565.5411324501038
flag = 0

start limit
75 33.997483015060425
flag = 1

limit end
limit: 566.6466581821442
flag = 0

start limit
75 32.796908140182495
flag = 1

limit end
limit: 567.1012465953827
flag = 0



KeyboardInterrupt: 

In [45]:
def ngram_mean_by_year(synset, corpus, startYear, endYear, smoothing, caseInsensitive):
    m = 0
    synset = wn.synset(synset[8:-2])
    for i in synset.lemmas():
        query = i.name()
        query = query.replace('_',' ') # recovery phrase from _
        params = dict(content=query, year_start=startYear, year_end=endYear,
                      corpus=corpora[corpus], smoothing=smoothing,
                      case_insensitive=caseInsensitive)
        if params['case_insensitive'] is False:
            params.pop('case_insensitive')
        if '?' in params['content']:
            params['content'] = params['content'].replace('?', '*')
        if '@' in params['content']:
            params['content'] = params['content'].replace('@', '=>')
        req = requests.get('https://books.google.com/ngrams/graph', params=params)
        res = re.findall('ngrams.data = (.*?);\\n', req.text)
        if res:
            data = {qry['ngram']: qry['timeseries'] for qry in literal_eval(res[0])}
            df = pd.DataFrame(data)
            df.insert(0, 'year', list(range(startYear, endYear + 1)))
        else:
            df = pd.DataFrame()
        if df.shape[1] > 1: # if the query exists in google ngrams.
            ngramsum = df.iloc[:,1].sum(axis = 0, skipna = True) # sum of percentage of the lemmas each year
            time.sleep(10) # to avoid defense from the google ngram server
            m += ngramsum
        else:
            m += 0 # irrelevant, obviously.
    return m/(endYear-startYear+1)

In [50]:
# check time
# original
original_start = time.time()
# GS_adopt_test['ngram_2009_2019'] = GS_adopt_test['Synsets'].apply(lambda syn: ngram_mean_by_year(syn, 'eng_2019', 2009, 2019, 0, True))
original_stop = time.time()
original_period = original_stop - original_start

string = 'original sleep time is ' + str(original_period) + '\n'
with open('time_checker.txt','a+') as f:
    f.write(string)

update_start = time.time()
# GS_adopt_test['ngram_2009_2019'] = GS_adopt_test['Synsets'].apply(lambda syn: ngram_mean_by_year_sleep(syn, 'eng_2019', 2009, 2019, 0, True))
update_stop = time.time()
update_period = update_stop - update_start
string = 'update sleep time is ' + str(update_period) + '\n'
with open('time_checker.txt','a+') as f:
    f.write(string)